In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
import os,sys


In [2]:
# Check current working directory
os.getcwd()

'/Users/yeonjoo/Downloads/Unilateral_foot'

In [116]:
# Change directory if necessary
#newdirectory = input("Path: ")
#os.chdir(newdirectory)
#os.getcwd()

In [4]:
# File import by filename
# Need subject and trial information
Subject = input("Subject: ")
Trial = input("Trial: ")
filename = Subject + "Trial"+ Trial

df = pd.read_csv(filename +'.csv')
df.head()

Subject: B01
Trial: 01B


,LHEEX,LHEEY,LHEEZ,LTOEX,LTOEY,LTOEZ,RHEEX,RHEEY,RHEEZ,RTOEX,RTOEY,RTOEZ
0,213.487488,237.421875,448.451904,152.920929,211.413589,268.553162,-42.172558,291.961273,233.400925,-237.794388,320.854614,214.888504
1,206.448959,237.571869,448.444214,143.744415,209.842484,269.410187,-36.156570,292.136383,233.808090,-231.766464,321.187836,215.134430
2,197.450485,237.683365,447.939270,131.803299,207.946152,270.171112,-28.587440,292.350067,234.323608,-224.168304,321.603699,215.438858
3,186.790955,237.709122,446.701294,117.349838,205.832184,270.662994,-19.769789,292.589478,234.928970,-215.298050,322.083527,215.786713
4,174.854614,237.606277,444.523987,100.744469,203.620560,270.738037,-10.068398,292.839020,235.602142,-205.517883,322.605591,216.160553


In [5]:
# Get Heel and Toe data for left and right sides
col_names = ['LHEEX','RHEEX', 'LTOEX', 'RTOEX']
name_rows_dict1 = {}

for col in col_names:
    vel_rows = []
    for idx, value in enumerate(df[col]):
        if idx == 0:
            vel_rows.append(0)
        else:
            vel_row = (df.loc[idx,col] - df.loc[idx-1,col])*100
            vel_rows.append(vel_row)
    name_rows_dict1[col] = vel_rows

vel_df = pd.DataFrame(name_rows_dict1)
vel_df.head()

,LHEEX,RHEEX,LTOEX,RTOEX
0,0.0000,0.0000,0.0000,0.0000
1,-703.8529,601.5988,-917.6514,602.7924
2,-899.8474,756.9130,-1194.1116,759.8160
3,-1065.9530,881.7651,-1445.3461,887.0254
4,-1193.6341,970.1391,-1660.5369,978.0167


In [6]:
# Identify Left Heel Strike index
# Heel Strike identified 
name_rows_dict2 = {}

lstrike_rows = []
for idx, value in enumerate(vel_df['LHEEX']):
    if vel_df.loc[idx, 'LHEEX'] <= 0:
        idx = idx + 1 
    else:
        if vel_df.loc[idx-1, 'LHEEX'] >= 0:
            idx = idx +1
        else:
            lstrike_rows.append(idx-1)
name_rows_dict2['LHSidx'] = lstrike_rows

lstrike_df = pd.DataFrame(name_rows_dict2)
lstrike_df.head()

,LHSidx
0,31
1,140
2,248
3,358
4,468


In [7]:
# Identify Left Heel Strike index
LHSval = pd.DataFrame(df.loc[lstrike_df['LHSidx'], 'LHEEX'])
LHSval_reidx = LHSval.reset_index().drop(columns=['index'])

lstrike_df['LHSval'] = LHSval_reidx['LHEEX']
lstrike_df.head()

,LHSidx,LHSval
0,31,-273.187195
1,140,-260.397949
2,248,-268.630615
3,358,-284.736816
4,468,-294.608337


In [301]:
name_rows_dict3 = {}

rstrike_rows = []
for idx, value in enumerate(vel_df['RHEEX']):
    if vel_df.loc[idx, 'RHEEX'] <= 0:
        idx = idx + 1 
    else:
        if vel_df.loc[idx-1, 'RHEEX'] >= 0:
            idx = idx +1
        else:
            rstrike_rows.append(idx)
            
name_rows_dict3['RHSidx'] = rstrike_rows

rstrike_df = pd.DataFrame(name_rows_dict3)
rstrike_df.head()

,RHSidx
0,37
1,138
2,236
3,337
4,437


In [302]:
RHSval = pd.DataFrame(df.loc[rstrike_df['RHSidx'], 'RHEEX'])
RHSval_reidx = RHSval.reset_index().drop(columns=['index'])

rstrike_df['RHSval'] = RHSval_reidx['RHEEX']
rstrike_df.head()

,RHSidx,RHSval
0,37,-382.269135
1,138,-356.461792
2,236,-343.386597
3,337,-337.019653
4,437,-282.524323


In [303]:
strike_df = pd.merge(lstrike_df, rstrike_df, left_index = True, right_index = True)
strike_df.head()

,LHSidx,LHSval,RHSidx,RHSval
0,88,-320.843018,37,-382.269135
1,187,-354.436890,138,-356.461792
2,288,-300.819672,236,-343.386597
3,389,-293.258118,337,-337.019653
4,488,-275.643219,437,-282.524323


In [304]:
# Calculate step time
Rsteptime = []
Lsteptime = []

for idx in strike_df.index:
    if strike_df.loc[0, 'LHSidx'] < strike_df.loc[0, 'RHSidx']:
        if idx == 0:
            Rstep_time = strike_df.loc[idx, 'RHSidx'] - strike_df.loc[idx, 'LHSidx']
            Rsteptime.append(Rstep_time)
            Lsteptime.append(0)
        else:
            Rstep_time = strike_df.loc[idx, 'RHSidx'] - strike_df.loc[idx, 'LHSidx']
            Rsteptime.append(Rstep_time)
            Lstep_time = strike_df.loc[idx, 'LHSidx'] - strike_df.loc[idx - 1,'RHSidx']
            Lsteptime.append(Lstep_time)
    else:
        if idx ==0:
            Rsteptime.append(0)
            Lstep_time = strike_df.loc[idx, 'LHSidx'] - strike_df.loc[idx,'RHSidx']
            Lsteptime.append(Lstep_time)
        else:
            Rstep_time = strike_df.loc[idx, 'RHSidx'] - strike_df.loc[idx - 1, 'LHSidx']
            Rsteptime.append(Rstep_time)
            Lstep_time = strike_df.loc[idx, 'LHSidx'] - strike_df.loc[idx,'RHSidx']
            Lsteptime.append(Lstep_time)
        

strike_df['Rstep_time'] = Rsteptime
strike_df['Lstep_time'] = Lsteptime

strike_df.head()

,LHSidx,LHSval,RHSidx,RHSval,Rstep_time,Lstep_time
0,88,-320.843018,37,-382.269135,0,51
1,187,-354.436890,138,-356.461792,50,49
2,288,-300.819672,236,-343.386597,49,52
3,389,-293.258118,337,-337.019653,49,52
4,488,-275.643219,437,-282.524323,48,51


In [305]:
# Get Treadmil Belt Speed
input_val = pd.read_csv('input_val.csv')
input_val = input_val.set_index('Unnamed: 0')
beltspeed = input_val.loc[Subject, 'belt_speed']
beltspeed, type(beltspeed)

(1.07, numpy.float64)

In [307]:
Rsteplength = []
Lsteplength = []

for idx in strike_df.index:
    if strike_df.loc[0, 'LHSidx'] < strike_df.loc[0, 'RHSidx']:
        if idx == 0:
            Rstep_length = strike_df.loc[idx, 'RHSval'] - (strike_df.loc[idx, 'LHSval'] - (beltspeed*strike_df.loc[idx, 'Rstep_time']*10))
            Rsteplength.append(Rstep_length)
            Lsteplength.append(0)
        else:
            Rstep_length = strike_df.loc[idx, 'RHSval'] - (strike_df.loc[idx, 'LHSval'] - (beltspeed*strike_df.loc[idx, 'Rstep_time']*10))
            Rsteplength.append(Rstep_length)
            Lstep_length = strike_df.loc[idx, 'LHSval'] - (strike_df.loc[idx - 1,'RHSval']-(beltspeed*strike_df.loc[idx, 'Lstep_time']*10))
            Lsteplength.append(Lstep_length)
    else:
        if idx ==0:
            Rsteplength.append(0)
            Lstep_length = strike_df.loc[idx, 'LHSval'] - (strike_df.loc[idx,'RHSval']-(beltspeed*strike_df.loc[idx, 'Lstep_time']*10))
            Lsteplength.append(Lstep_length)
        else:
            Rstep_length = strike_df.loc[idx, 'RHSval'] - (strike_df.loc[idx - 1, 'LHSval'] - (beltspeed*strike_df.loc[idx, 'Rstep_time']*10))
            Rsteplength.append(Rstep_length)
            Lstep_length = strike_df.loc[idx, 'LHSval'] - (strike_df.loc[idx,'RHSval']-(beltspeed*strike_df.loc[idx, 'Lstep_time']*10))
            Lsteplength.append(Lstep_length)
        

strike_df['Rstep_length'] = Rsteplength
strike_df['Lstep_length'] = Lsteplength

strike_df.head()

,LHSidx,LHSval,RHSidx,RHSval,Rstep_time,Lstep_time,Rstep_length,Lstep_length
0,88,-320.843018,37,-382.269135,0,51,0.000000,607.126117
1,187,-354.436890,138,-356.461792,50,49,499.381226,526.324902
2,288,-300.819672,236,-343.386597,49,52,535.350293,598.966925
3,389,-293.258118,337,-337.019653,49,52,488.100019,600.161535
4,488,-275.643219,437,-282.524323,48,51,524.333795,552.581104
5,589,-270.323303,537,-314.384979,49,52,485.558240,600.461676
6,691,-294.073822,639,-286.261383,50,52,519.061920,548.587561
7,792,-323.974670,739,-348.638580,48,53,459.035242,591.763910
8,893,-362.938202,842,-373.635590,50,51,485.339080,556.397388
9,994,-380.154175,943,-383.267914,50,51,514.670288,548.813739


In [309]:
strike_df.to_csv('G02Trial02step.csv')

In [308]:

step = ['Rstep_time','Rstep_length','Lstep_time','Lstep_length']
rows = []

for col in step:
    df_attr = getattr(strike_df, col)
    row = pd.DataFrame({filename+"_Avg":[df_attr.mean()], 
                        filename+"_SD":[df_attr.std()]
                       }, index = [col])
    rows.append(row)

step_df = pd.concat(rows)
step_df

,G02Trial02A50_Avg,G02Trial02A50_SD
Rstep_time,-34.890365,35.910137
Rstep_length,-393.963583,390.297564
Lstep_time,134.199336,34.988571
Lstep_length,1456.696877,379.126689


In [268]:
if not os.path.exists('step.csv'):
    step_df.to_csv('step.csv', index=step, mode='w')
else:
    step_df.to_csv('step.csv', index=step, mode='a')

In [246]:
#step_df.to_csv('./step.csv', index=False)
#step = open('step.csv',"w")
#step

<_io.TextIOWrapper name='step.csv' mode='w' encoding='UTF-8'>

In [248]:
#step_df2 = pd.concat([step, step_df], axis=1)
#step_df2